In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jaxtrees
from jax.random import PRNGKey, split
import jax
from jax import numpy as jnp
from jaxtrees.models import UpdateModel
from jaxtrees.models.functional import pass_up
from jaxtrees.execution import RedBlackExecutor
from jaxtrees.mcmc import ParameterStore, FixedParameter
import jax
from functools import partial

In [3]:
key = PRNGKey(0)

In [4]:
tree = jaxtrees.tree.builders.THeight_legacy(3,2)
subkey, key = split(key)
noise_tree = jaxtrees.tree.initializers.initialize_noise(tree, key, (2,))

In [5]:
class MCMCExample(UpdateModel):
    def update(self, parent_value,children_values,node_value, parameters):
        if not children_values:
            return {'noise': parent_value['noise']}, True
        if not parent_value:
            return {'noise': children_values['noise'].mean(0)}, True
        
        parent_noise = parent_value['noise']
        children_noise = children_values['noise'].mean(0)

        result = (1-parameters['lambd'])*parent_noise+parameters['lambd'] *children_noise

        return {'noise': result}, True
    

In [6]:
params = ParameterStore({
    'lambd': FixedParameter(0.5),
    # put more parameters here
})

In [7]:
model = MCMCExample()
exe = RedBlackExecutor(model)

In [8]:
it = exe.get_iterator(noise_tree)

for node in it:
    key, subkey = jax.random.split(key)
    proposed = params.propose(subkey)

    accepted = exe.update(node, proposed)
    params.update(proposed, accepted)